In [698]:
import pandas as pd
from sdv.lite import TabularPreset
from sdv.evaluation import evaluate
import math

In [699]:
config_file = "config.csv"
config = pd.read_csv(config_file)
data_file = config["Input File"][0]
target_file = config["Output File"][0]
data = pd.read_csv(data_file)
headers = data.columns
features = config["Features"].to_list()
allowed_percentage = config["Percentage"][0]
number_of_samples = int(config["Number of Samples"][0])
types = config["Type"].to_list()
sub_types = config["Subtype"].to_list()
primary_key = config["Primary Key"][0]

In [700]:

#Remove unspecified columns
for header in headers:
    if not header in features:
        data.pop(header)
        #print("Deleting column", header, "because of configuration.")

#Remove columns with to many NaNs
headers = data.columns
for header in headers:
    nan_percentage = data[header].isna().sum()/len(data[header].index)
    if nan_percentage > allowed_percentage:
        data.pop(header)
        headers = headers.drop(header)
        print("Deleting column", header, "because of" ,"{:.2%}".format(nan_percentage), "NaNs.")

print(headers)

Deleting column Mutation Count because of 0.23% NaNs.
Deleting column Sample Type because of 2.56% NaNs.
Deleting column Neoplasm Histologic Grade because of 91.28% NaNs.
Deleting column Treatment Details before PDX because of 90.58% NaNs.
Index(['Cancer Type Detailed', 'Oncotree Code', 'Overall Survival (Months)',
       'Diagnosis Age', 'TERT', 'IDH1', 'PTEN'],
      dtype='object')


In [701]:
#Generating Metadata

#metadata = {}
#metadata["fields"] = {}
#for i in range(len(features)):
#    if features[i] in headers:
#        metadata["fields"][features[i]] = {}
#        metadata["fields"][features[i]]["type"] = types[i]
#        if sub_types[i] == sub_types[i]:
#            metadata["fields"][features[i]]["subtype"] = sub_types[i]

#metadata["constraints"] = []

#print(metadata)



{'fields': {'Cancer Type Detailed': {'type': 'categorical'}, 'Oncotree Code': {'type': 'categorical'}, 'Overall Survival (Months)': {'type': 'numerical', 'subtype': 'float'}, 'Diagnosis Age': {'type': 'numerical', 'subtype': 'integer'}, 'TERT': {'type': 'categorical'}, 'IDH1': {'type': 'categorical'}, 'PTEN': {'type': 'categorical'}}, 'constraints': []}


In [702]:
#fit a model to the data and generating data
#model = TabularPreset(name='FAST_ML', metadata=metadata)
model = TabularPreset(name='FAST_ML')

model.fit(data)

new_data = model.sample(num_rows=number_of_samples)

new_data = new_data.round(decimals = 2)

new_data.to_csv(target_file, index=False)

print("The generated dat is", "{:.2%}".format(evaluate(new_data, data, metrics=['CSTest'])), "accurate to the original data. If this is unsatisfactory try using less or different features, more data or removing more NaNs.")

ValueError: There are non-numerical values in your data.